In [9]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:85% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
""")) 

<b><font color="red" size="5">ch.14 웹데이터 수집</font></b>
# 1절. BeautifulSoup과 parser
```pip insatall bs4``` 아나콘다를 설치하면 7500여개의 패키지가 설치됨.
- 공식 사이트 : https://www.crummy.com/software/BeautifulSoup/
- Docs : https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [10]:
import requests # HTTP 요청처리 lib
from requests_file import FileAdapter

In [29]:
s = requests.Session() # HTTP 요청관리를 위한 세션 객체
s.mount("file://", FileAdapter())
response = s.get('file:///AI_x/04_source/01_python/data/ch14_sample.html')
response

<Response [200]>

In [12]:
response.status_code 
# 200 : 정상
# 404 : 없는 페이지
# 406 : get, post 오류

200

In [15]:
response.content # 바이너리 형식의 내용

b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90 \xec\x95\x88\xec\x9d\x98 \xeb\x82\xb4\xec\x9a\xa9</div>\r\n  <p>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\xb3\xec\x97\x90\xec\x84\x9c \xed\x99\x9c\xec\x9a\xa9\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4</p>\r\n  <div class="contents">\r\n    \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\xa5\xbc \xec\x96\xb4\xeb\x96\xbb\xea\xb2\x8c \xec\x9e\x91\xec\x84\xb1\xed\x95\x98\xeb\x8a\x90\xeb\x83\x90\xec\x97\x90 \xeb\x94\xb0\xeb\x9d\xbc\r\n    <span>\xeb\x8b\xa4\xeb\xa5\xb8<b>\xec\x9a\x94\xec\x86\x8c\xea\xb0\x80 \xeb\xb0\x98\xed\x99\x98</b></span>\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4\r\n  </div>\r\n  <div>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\

In [16]:
response.content.decode('utf-8')

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [17]:
response.text

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [30]:
# html 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, # response.text
                    "html.parser")
# soup

In [34]:
# soup.select_one('선택자') : 해당 선택자 처음 하나만
el = soup.select_one('h1') # 처음 나오는 h1태그 하나만
print('el :', el)
print('el.text :', el.text)
print('el.string :', el.text)
print('el의 속성들 :', el.attrs)
print('el의 title속성 :', el.attrs['title'])
print('el의 title속성 :', el.attrs.get('title') )
print('el의 이름:', el.name)

el : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
el.text : Hello, CSS
el.string : Hello, CSS
el의 속성들 : {'class': ['greeting', 'css'], 'id': 'text', 'title': 'greeting'}
el의 title속성 : greeting
el의 title속성 : greeting
el의 이름: h1


In [39]:
# soup.select('선택자') : 해당 선택자 모든 엘리먼트를 리스트로
el = soup.select('h1')
print('리스트 el :', el)
print('el의 text들 :', [e.text for e in el])
# for e in el:
#    print(e.text, end=',')
print('el의 속성들 :', [e.attrs for e in el])
print('el의 class속성들 :', [e.attrs.get('class') for e in el])

리스트 el : [<h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
el의 text들 : ['Hello, CSS', 'Hi, CSS']
el의 속성들 : [{'class': ['greeting', 'css'], 'id': 'text', 'title': 'greeting'}, {'class': ['css']}]
el의 class속성들 : [['greeting', 'css'], ['css']]


In [48]:
# select_one(선택자)와 find(태그, 속성)
print("select_one :", soup.select_one('h1.css'))
print('find :', soup.find('h1', {'class':'css'}))
print('find :', soup.find('h1', class_='css'))
print()
print('select_one :', soup.select_one('h1#text'))
print('find :', soup.find('h1', {'id':'text'}));
print()
print('select')

select_one : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
find : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
find : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>

select_one : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
find : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>

select


In [56]:
# select(선택자)와 find_all(태그, 속성)
print('모든 h1.css, span 태그(select) :',
     soup.select('h1.css, span'))
print('모든 h1.css, span태그(find_all) :',
     soup.find_all(['h1','span'], class_='css'))

모든 h1.css, span 태그(select) : [<h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
모든 h1.css, span태그(find_all) : [<h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]


In [61]:
# 없는 엘리먼트 찾기
print('find_all(빈 list) :', soup.find_all('a', class_='css'))
print('find(None) :', soup.find('a', class_='css'))
print('select(빈 list) :', soup.select('a.css'))
print('select_one(None) :', soup.select_one('a.css'))

find_all(빈 list) : []
find(None) : None
select(빈 list) : []
select_one(None) : None


# 2절. 정적 웹 데이터 수집(정적 웹크롤링)
- json, html, xml
## 2.1 JSON파일
- request모듈(get)
- urllib.request모듈(urlopen)

In [69]:
# 크롤링 허용 범위는 사이트마다 ~/robots.txt에서 확인할 수 있습니다.

In [64]:
# 방법 1
import requests
response = requests.get('http://api.github.com')
response, response.status_code

(<Response [403]>, 403)

In [131]:
# 방법2
from urllib.request import urlopen
#response = urlopen('http://api.github.com')
#response

In [128]:
text = '{"속성1":"값1","속성2":"값2"}'
type(text)

str

In [129]:
# 문자(딕셔너리 타입)를 딕셔너리
# '{"속성1":"값1","속성2":"값2"}' => {"속성1":"값1","속성2":"값2"}
import json
json.loads(text)

{'속성1': '값1', '속성2': '값2'}

## 2.2 html 파일
### 1) 환율정보 가져오기(네이버->증권->시장지표)
- https://finance.naver.com/marketindex/

In [77]:
# 방법 1
import requests
from bs4 import BeautifulSoup
url = 'https://finance.naver.com/marketindex'
response = requests.get(url)
# response, response.status_code
# response.content
# response.content.decode('cp949') == response.text
soup = BeautifulSoup(response.text, # response.content
                    'html.parser')

In [83]:
# 방법2
from urllib.request import urlopen
response = urlopen(url)
# response.status 상태코드
# response.read()
# response.read().decode('cp949')
soup = BeautifulSoup(response, # response.read()
                     'html.parser')

In [86]:
title = soup.select('h3.h_lst > span.blind')
for t in title:
    print(t.text, end='\t')

미국 USD	일본 JPY(100엔)	유럽연합 EUR	중국 CNY	달러/일본 엔	유로/달러	영국 파운드/달러	달러인덱스	WTI	휘발유	국제 금	국내 금	

In [102]:
price = soup.select('div.head_info > span.value')
[p.text for p in price]
[round(float(p.text.replace(',',''))) for p in price]
[round(float(''.join(p.text.split(',')))) for p in price]

[1375, 953, 1556, 191, 144, 1, 1, 99, 61, 1633, 3300, 146218]

In [92]:
out = '1,374.70'
round(float(''.join(out.split(','))))

1375

In [95]:
round(float(out.replace(',','')))

1375

In [104]:
unit = soup.select('div.head_info > span > span.blind')
unit = [u.text for u in unit]
unit.insert(7, '') # 7번째 index에 ''추가
unit

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [111]:
status = soup.select('div.head_info > span.blind')
[t.text for t in title]
[p.text for p in price]
unit
[s.string for s in status]

['하락', '하락', '하락', '하락', '상승', '하락', '하락', '상승', '하락', '하락', '하락', '상승']

In [112]:
len(title), len(price), len(unit), len(status)

(12, 12, 12, 12)

In [115]:
for idx in range(len(title)):
    print("{}. {} : {}{} - {}".format(idx+1,
                                 title[idx].text,
                                 price[idx].text,
                                 unit[idx],
                                 status[idx].text))

1. 미국 USD : 1,375.20원 - 하락
2. 일본 JPY(100엔) : 953.05원 - 하락
3. 유럽연합 EUR : 1,556.38원 - 하락
4. 중국 CNY : 191.04원 - 하락
5. 달러/일본 엔 : 144.3800엔 - 상승
6. 유로/달러 : 1.1342달러 - 하락
7. 영국 파운드/달러 : 1.3520달러 - 하락
8. 달러인덱스 : 99.4200 - 상승
9. WTI : 60.89달러 - 하락
10. 휘발유 : 1633.17원 - 하락
11. 국제 금 : 3300.4달러 - 하락
12. 국내 금 : 146218.18원 - 상승


In [122]:
for t, p, u, s  in zip(title, price, unit, status):
    print("{} : {}{} - {}".format(t.text,
                                 p.text,
                                 u,
                                 s.text))

미국 USD : 1,375.20원 - 하락
일본 JPY(100엔) : 953.05원 - 하락
유럽연합 EUR : 1,556.38원 - 하락
중국 CNY : 191.04원 - 하락
달러/일본 엔 : 144.3800엔 - 상승
유로/달러 : 1.1342달러 - 하락
영국 파운드/달러 : 1.3520달러 - 하락
달러인덱스 : 99.4200 - 상승
WTI : 60.89달러 - 하락
휘발유 : 1633.17원 - 하락
국제 금 : 3300.4달러 - 하락
국내 금 : 146218.18원 - 상승


### 2) 이번주 로또번호 출력
- https://dhlottery.co.kr/gameResult.do?method=byWin(google에 "로또번호 당첨번호" 검색)
```
    1173회(2025년 05월 24일 추첨)
    당첨번호 [1 5 18 20 30 35]
    보너스 3
```

In [135]:
# 방법 1의 soup객체 생성
import requests
from bs4 import BeautifulSoup
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = requests.get(url)
soup = BeautifulSoup(response.text, # response.text
                    'html.parser')

In [141]:
# 방법2의 sou객체 생성
from urllib.request import urlopen
response = urlopen(url)
soup = BeautifulSoup(response, "html.parser")

In [143]:
times = soup.select_one('div.win_result > h4 > strong').text # 1173회
date = soup.select_one('div.win_result >p.desc').text  #(2025년 05월 24일 추첨)
print(times, date)
title1 = soup.select_one('div.num.win > strong').text
lotto_number = soup.select('div.num.win > p > span')
print(title1, [int(lotto.text) for lotto in lotto_number])
title2 = soup.select_one('div.num.bonus > strong').string
bonus_number = soup.select_one('div.num.bonus > p > span').text
print(title2, bonus_number)

1173회 (2025년 05월 24일 추첨)
당첨번호 [1, 5, 18, 20, 30, 35]
보너스 3


In [153]:
# 위의 select 함수를 find함수로
# times = soup.select_one('div.win_result > h4 > strong').text # 1173회
win_result = soup.find('div', class_="win_result")
times = win_result.find('strong').text
# date = soup.select_one('div.win_result >p.desc').text  #(2025년 05월 24일 추첨)
date = soup.find('p', class_='desc').text
print(times, date)
# title1 = soup.select_one('div.num.win > strong').text
num_win = soup.find('div', class_=['num', 'win'])
title1 = num_win.find('strong').text
# lotto_number = soup.select('div.num.win > p > span')
lotto_number = num_win.find_all('span') #배열
print(title1, [int(lotto.text) for lotto in lotto_number])
# title2 = soup.select_one('div.num.bonus > strong').string
num_bonus = soup.find('div', class_=['num','bonus'])
title2 = num_bonus.find('strong').text
# bonus_number = soup.select_one('div.num.bonus > p > span').text
bonus_number = num_bonus.find('span').text
print(title2, bonus_number)

1173회 (2025년 05월 24일 추첨)
당첨번호 [1, 5, 18, 20, 30, 35]
당첨번호 1


### 3) 다음 검색 리스트
```
no title link
0 [비트코인 2025] 백악관 크립토 차르 “美 정부 비트코인 추가 매입 검토…부채 안 늘리면 가능 https://v.daum.net/v/20250528103907230
1. [비즈 나우] 비트코인 2025 컨퍼런스 개막…'전략자산' 선언 코앞 https://v.daum.net/v/20250528075215864
....
````

In [199]:
# 방법1
import requests
from bs4 import BeautifulSoup
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q='+word
response = requests.get(url)
soup = BeautifulSoup(response.text, # response.content
                    "html.parser")
items_find_list = [] # 검색한 결과를 담을 dict list
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
# len(items_el)
for idx, item in enumerate(items_el):
    items_find_list.append({'no': idx,
                           'title': item.text,
                           'link': item.attrs['href']})
import pandas as pd
pd.DataFrame(items_find_list)

,no,title,link
0,0,美·日 국채 팔고 비트코인 샀다…'안전자산' 등극,http://v.daum.net/v/20250528154126316
1,1,"블랙록, 자사 비트코인 ETF 보유량 25% 확대…기관 투자 본격화 신호탄",http://v.daum.net/v/20250528151802080
2,2,[비트코인 2025] 백악관 크립토 차르 “美 정부 비트코인 추가 매입 검토…부채...,http://v.daum.net/v/20250528103907230
3,3,[비즈 나우] 비트코인 2025 컨퍼런스 개막…'전략자산' 선언 코앞,http://v.daum.net/v/20250528075215864
4,4,“맥O날드보다 맛없어!” 혹평이 가득한 트럼프 만찬과 비트코인 피자데이[엠블록레터],http://v.daum.net/v/20250528143002735
5,5,"비트코인, 트럼프 미디어 비축 소식에도 주춤…1억5100만원대",http://v.daum.net/v/20250528095112897
6,6,"美 상원의원 ""트럼프 대통령, 비트코인법 지지""",http://v.daum.net/v/20250528090342853
7,7,[비트코인 2025] 로빈후드 창업자 “토큰화 증권은 美 ‘자본 패권’ 키우는 수단”,http://v.daum.net/v/20250528110600761
8,8,"트럼프미디어그룹, 25억 달러 규모 자금 조달 통해 비트코인 매입 예고",http://v.daum.net/v/20250528084245020
9,9,"'비트코인 빚투' 스트레티지, 또 샀다…보유량 58만개 돌파",http://v.daum.net/v/20250528042404292


In [177]:

import requests
from bs4 import BeautifulSoup
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q='+word
response = requests.get(url)
soup = BeautifulSoup(response.text, # response.content
                    "html.parser")
items_find_list = [] # 검색한 결과를 담을 2차원 리스트
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
for idx, item in enumerate(items_el):
    items_find_list.append([idx, item.text, item.attrs.get('href')])
# items_find_list
import pandas as pd
pd.DataFrame(items_find_list, columns=['no', 'title', 'link'])

%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


,no,title,link
0,0,[비트코인 2025] 백악관 크립토 차르 “美 정부 비트코인 추가 매입 검토…부채...,http://v.daum.net/v/20250528103907230
1,1,EU관세 발언 후 10만8000달러선 회복 비트코인…파키스탄 ‘비트코인 채굴’ 전...,http://v.daum.net/v/20250526080440640
2,2,"숨 고르는 비트코인, 10만8000달러선 '주춤'",http://v.daum.net/v/20250528095921258
3,3,[비즈 나우] 비트코인 2025 컨퍼런스 개막…'전략자산' 선언 코앞,http://v.daum.net/v/20250528075215864
4,4,"""비트코인 산다""…트럼프家, 25억달러 자금 조달 추진[코인브리핑]",http://v.daum.net/v/20250528111837527
5,5,"비트코인, 트럼프 미디어 비축 소식에도 주춤…1억5100만원대",http://v.daum.net/v/20250528095112897
6,6,"'비트코인 빚투' 스트레티지, 또 샀다…보유량 58만개 돌파",http://v.daum.net/v/20250528042404292
7,7,"10만 달러, 11만 달러… 트럼프 관세 변덕에 비트코인 '출렁'",http://v.daum.net/v/20250527132843430
8,8,'해킹 산업화'의 원흉 '비트코인'…위치추적 안 되자 활개[은폐⑥],http://v.daum.net/v/20250527090029503
9,9,"美 상원의원 ""트럼프 대통령, 비트코인법 지지""",http://v.daum.net/v/20250528090342853


In [201]:
# 다음 뉴스 검색(키워드, 원하는 페이지수)
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
keyword = '비트코인'
page = 2
# url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&q={keyword}&p={page}'
# response = requests.get(url)
url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y'
params = {'q':keyword, 'p':page}
response = requests.get(url, params=params)
soup = BeautifulSoup(response.text,  # response.content
                    'html.parser')
items_find_list = [] # 검색한 결과를 담을 dict list
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
# len(items_el)
for idx, item in enumerate(items_el):
    items_find_list.append({'no': (page-1)*10 + idx,
                           'title': item.text,
                           'link': item.attrs['href']})
    print({'no': (page-1)*10 + idx,
           'title': item.text,
           'link': item.attrs['href']})

{'no': 10, 'title': ' 달러 스테이블코인, 국내 거래 확산…당국 외환흐름 통제력 약화 우려 ', 'link': 'http://v.daum.net/v/20250528152520516'}
{'no': 11, 'title': ' 전력거래소, `공감·혁신` 토대 조직문화 바꾸기 ', 'link': 'http://v.daum.net/v/20250528152647580'}
{'no': 12, 'title': " 숨 고르는 비트코인, 10만8000달러선 '주춤' ", 'link': 'http://v.daum.net/v/20250528095921258'}
{'no': 13, 'title': ' [포토]라스베이거스에서 열린 2025 비트코인 콘퍼런스 ', 'link': 'http://v.daum.net/v/20250528155536962'}
{'no': 14, 'title': ' 연일 최고가 경신하는 비트코인…투자자들 강 건너 불구경하는 이유는 ', 'link': 'http://v.daum.net/v/20250526212400275'}
{'no': 15, 'title': ' "비트코인 산다"…트럼프家, 25억달러 자금 조달 추진[코인브리핑] ', 'link': 'http://v.daum.net/v/20250528111837527'}
{'no': 16, 'title': ' [김대호 박사의 오늘의 키워드] 소비자신뢰지수·비트코인 2025·머스크 한마디·애플 괘씸죄 ', 'link': 'http://v.daum.net/v/20250528071325361'}
{'no': 17, 'title': ' 비트코인과 엇갈리는 이더리움·리플, 왜? ', 'link': 'http://v.daum.net/v/20250526164319028'}
{'no': 18, 'title': ' 잘 나가는 비트코인 산다는데 왜?…트럼프 미디어 주가 10% 급락 [투자360] ', 'link': 'http://v.daum.net/v/20250528074002668'}
{'no': 19, 'titl

In [202]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
def collect_list(keyword, page=1):
    'keyword로 다음 검색한 결과(해당 page)를 return'
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y'
    params = {'q':keyword, 'p':page}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text,  # response.content
                    'html.parser')
    items_find_list = [] # 검색한 결과를 담을 dict list
    items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
    for idx, item in enumerate(items_el):
        items_find_list.append({'no': (page-1)*10 + idx,
                           'title': item.text,
                           'link': item.attrs['href']})
    return items_find_list

In [203]:
collect_list('대선투표',2)

[{'no': 10,
  'title': " 공약집 없이 대선 투표 시작… 가열되는 '비방전' 투표율 낮출라 ",
  'link': 'http://v.daum.net/v/20250524123705877'},
 {'no': 11,
  'title': " '시장 스타일 구기는 것 쯤이야'…광명시, 대선투표 독려 영상 화제 ",
  'link': 'http://v.daum.net/v/20250528100556682'},
 {'no': 12,
  'title': " 국민의힘 방세환 광주시장, 대선 투표참여 독려 '눈길' ",
  'link': 'http://v.daum.net/v/20250527155213156'},
 {'no': 13,
  'title': ' 전남도, 대선 투표 참여 캠페인 ',
  'link': 'http://v.daum.net/v/20250528112007618'},
 {'no': 14,
  'title': ' 대선 투표참여 홍보에 발벗고 나선 김영록 지사 ',
  'link': 'http://v.daum.net/v/20250527144205460'},
 {'no': 15,
  'title': ' <호텔소식> 센텀비즈니스호텔 "대선 투표 인증샷, 당일 선착순 1박 무료" ',
  'link': 'http://v.daum.net/v/20250527112515888'},
 {'no': 16,
  'title': ' ‘대선’ 투표 참여 독려하는 강북구…홍보 현수막·배너 설치 ',
  'link': 'http://v.daum.net/v/20250527141101069'},
 {'no': 17,
  'title': ' 김보라 안성시장 “제21대 대선 투표 꼭 하세요”…유권자 독려 ',
  'link': 'http://v.daum.net/v/20250526171720742'},
 {'no': 18,
  'title': ' [동정] 권익현 부안군수, 대선 투표 독려 캠페인 동참 ',
  'link': 'http://v.daum.net/v/20250528135

In [204]:
# 원하는 keyword로 다음검색
result = []
keyword = "청바지"
pages = 4
for page in range(1, pages+1):
    result.extend(collect_list(keyword, page))
    time.sleep(1)
result_df = pd.DataFrame(result)
result_df.head()

,no,title,link
0,0,"'이병헌♥' 이민정, 매장 통째로 빌려 청바지 쇼핑",http://v.daum.net/v/20250528050507783
1,1,"“추앙받을 만 하네”… 김지원, 셔츠에 청바지만 입어도 공항 ‘정지’",http://v.daum.net/v/20250526182411395
2,2,흰 티셔츠에 청바지 정석대로 입는 법,http://v.daum.net/v/20250527185425864
3,3,"이민정, 매장 빌려 폭풍 쇼핑 ""10년 전 샀던 청바지는 응급실""(MJ)",http://v.daum.net/v/20250527191015309
4,4,"‘이병헌♥’ 이민정, 청바지 사는데 매장 통째로 빌렸다..남다른 쇼핑 클래스 “나...",http://v.daum.net/v/20250527192310643


In [206]:
keywords = ['청바지','면바지']
pages = 5
result0 = [] # 청바지 검색결과 50개
result1 = [] # 면바지 검색결과 50개
for i, keyword in enumerate(keywords):
    for page in range(1, pages+1):
        print(f'~ ~ ~ {i}번째 {keyword} {page} 검색 중입니다 ~ ~ ~')
        if i==0:
            result0.extend(collect_list(keyword, page))
        else:
            result1.extend(collect_list(keyword, page))
        time.sleep(4)

~ ~ ~ 0번째 청바지 1 검색 중입니다 ~ ~ ~
~ ~ ~ 0번째 청바지 2 검색 중입니다 ~ ~ ~
~ ~ ~ 0번째 청바지 3 검색 중입니다 ~ ~ ~
~ ~ ~ 0번째 청바지 4 검색 중입니다 ~ ~ ~
~ ~ ~ 0번째 청바지 5 검색 중입니다 ~ ~ ~
~ ~ ~ 1번째 면바지 1 검색 중입니다 ~ ~ ~
~ ~ ~ 1번째 면바지 2 검색 중입니다 ~ ~ ~
~ ~ ~ 1번째 면바지 3 검색 중입니다 ~ ~ ~
~ ~ ~ 1번째 면바지 4 검색 중입니다 ~ ~ ~
~ ~ ~ 1번째 면바지 5 검색 중입니다 ~ ~ ~


In [207]:
result0_df = pd.DataFrame(result0)
result0_df.head()

""


In [208]:
result1_df = pd.DataFrame(result1)
result1_df.head()

""


In [209]:
result0_df.to_csv('data/ch14_'+keywords[0]+'.csv', index=False, encoding='cp949')
result1_df.to_csv('data/ch14_'+keywords[1]+'.csv', index=False)

-꼭 User-Agent를 사용하여야 하는 경우 : https://www.melon.com/chart/

In [211]:
# 방법 1
import requests
from bs4 import BeautifulSoup
url = 'https://www.melon.com/chart/'
melonpage = requests.get(url)
melonpage.status_code

406

In [213]:
# 방법 2
from urllib.request import urlopen, Request
# melonpage = urlopen(url) 에러남

In [215]:
# User-Agent 추가
headers = {'user-agent':
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36'
}
# 방법2
request = Request(url, headers=headers)
melonpage = urlopen(request)
# 방법1
melonpage = requests.get(url, headers=headers)
soup = BeautifulSoup(melonpage.text, #melonpage.content
                    "html.parser")
#soup

In [224]:
# 멜론 순위, 노래제목, 가수, 좋아요 갯수
# 1위 | 너에게 닿기를 | 10cm (100위까지 크롤링)
title_els = soup.select('div.ellipsis.rank01 > span')
singer_els = soup.select('div.ellipsis.rank02 > span.checkEllipsis')
len([title.text.strip() for title in title_els]), \
len([singer.text for singer in singer_els])
for idx, (title, singer) in enumerate(zip(title_els, singer_els)):
    print('{}위 | {} - {}'.format(idx+1,
                                title.text.strip(),
                                singer.text))

1위 | 너에게 닿기를 - 10CM
2위 | Never Ending Story - 아이유
3위 | Drowning - WOODZ
4위 | like JENNIE - 제니 (JENNIE)
5위 | 모르시나요(PROD.로코베리) - 조째즈
6위 | THUNDER - 세븐틴 (SEVENTEEN)
7위 | TOO BAD (feat. Anderson .Paak) - G-DRAGON
8위 | 네모의 꿈 - 아이유
9위 | Whiplash - aespa
10위 | HOME SWEET HOME (feat. 태양, 대성) - G-DRAGON
11위 | 나는 반딧불 - 황가람
12위 | 어제보다 슬픈 오늘 - 우디 (Woody)
13위 | 오늘만 I LOVE YOU - BOYNEXTDOOR
14위 | REBEL HEART - IVE (아이브)
15위 | HAPPY - DAY6 (데이식스)
16위 | 빨간 운동화 - 아이유
17위 | Flower - 오반(OVAN)
18위 | APT. - 로제 (ROSÉ), Bruno Mars
19위 | 청춘만화 - 이무진
20위 | HOT - LE SSERAFIM (르세라핌)
21위 | 10월 4일 - 아이유
22위 | 한 페이지가 될 수 있게 - DAY6 (데이식스)
23위 | Last Scene (Feat. 원슈타인) - 아이유
24위 | HBD - 세븐틴 (SEVENTEEN)
25위 | MY LOVE(2025) - 이예은, 아샤트리, 전건호
26위 | 그날이 오면 - 투모로우바이투게더
27위 | 내게 사랑이 뭐냐고 물어본다면 - 로이킴
28위 | Fly Up - RIIZE
29위 | Welcome to the Show - DAY6 (데이식스)
30위 | 소나기 - 이클립스 (ECLIPSE)
31위 | 천상연 - 이창섭
32위 | Die With A Smile - Lady Gaga, Bruno Mars
33위 | toxic till the end - 로제 (ROSÉ)
34위 | HANDS UP - MEOVV (미야오)
35위 | Superno

### 4) User-agent를 추가하여 크롤링
- urlopen()함수를 사용하면 크롤링이 안 되는 사이트
- User-agent를 추가하여 

In [200]:
# 방법2
from urllib.request import urlopen
import urllib.parse
word = '비트코인'
word = urllib.parse.quote(word)
# print(word)
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q='+word
response = urlopen(url)
soup = BeautifulSoup(response, # response.content
                    "html.parser")
items_find_list = [] # 검색한 결과를 담을 dict list
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
# len(items_el)
for idx, item in enumerate(items_el):
    items_find_list.append({'no': idx,
                           'title': item.text,
                           'link': item.attrs['href']})
import pandas as pd
pd.DataFrame(items_find_list)

""
